# 0. The Obligatory Part

In [ ]:
# Run this cell if data in Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Install ortools
!pip3 install ortools

In [1]:
# Import library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from competency_assessment import CompetencyAssessment
from ortools.sat.python import cp_model

# 1. Define the Data Structure

In [ ]:
# Data path
# new_employee_path = '/content/drive/MyDrive/Tadika Mesra Bunga Matahari/#1 Optimization Problem/project1_task-assignment/data/new_employee.csv'
# new_task_path = '/content/drive/MyDrive/Tadika Mesra Bunga Matahari/#1 Optimization Problem/project1_task-assignment/data/new_task_1.csv'

In [2]:
# Run this if the data in Local/Repository
new_employee_path = '../../data/fixed_employee_data.csv'
new_task_path = '../../data/task_data.csv'

## 1.1. Pre-Processing: Employee Data

In [3]:
# Read data
employee_skills_df = pd.read_csv(new_employee_path, index_col='employee_id')
employee_skills_df.drop(columns=['No', 'Role'], inplace=True, errors='ignore')

employees = employee_skills_df.index.tolist()
skills_name = employee_skills_df.columns[1:].tolist()

employee_skills_df

,Mathematics.Linear Algebra,Mathematics.Differential Equations,Mathematics.Optimization Technique,Mathematics.Calculus,Mathematics.Combinatorics & Graph,Statistics & Probabilities.Statistics,Statistics & Probabilities.Probability & Sampling,Statistics & Probabilities.Bayessian Method,Statistics & Probabilities.Hypothesis Testing,Statistics & Probabilities.A/B Testing,...,Data & Cloud Engineering.Data Security & Privacy,Data & Cloud Engineering.Distributed Systems,"Data & Cloud Engineering.Monitoring, Logging, Alerting",MLOPS.ML Lifecycle,MLOPS.ML Model Pipeline,MLOPS.Model Versioning,MLOPS.Automated Testing & Validation,MLOPS.Scalable Model Training,"MLOPS.Model Deployment, Monitoring, Logging",MLOPS.Ethical AI & Bias Mitigation
employee_id,,,,,,,,,,,,,,,,,,,,,
Talent 1,1,1,2,1,2,3,3,2,2,2,...,1,1,1,2,1,1,1,1,1,2
Talent 2,4,4,4,4,3,5,4,3,3,3,...,3,2,1,1,1,1,1,1,1,1
Talent 3,3,2,2,3,2,3,3,2,2,3,...,3,3,3,3,2,3,2,3,3,2
Talent 4,3,2,3,2,3,4,3,2,4,2,...,3,1,1,2,2,2,2,2,2,2
Talent 5,2,1,3,1,1,1,2,1,1,2,...,2,1,1,3,2,5,2,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Talent 107,1,4,4,3,4,3,4,4,4,4,...,5,5,5,3,5,3,4,3,4,2
Talent 108,3,3,3,3,4,2,2,2,2,3,...,4,3,4,2,3,3,2,2,3,2
Talent 109,5,5,4,5,3,4,4,3,3,3,...,5,5,5,5,5,5,5,5,5,5


## 1.2. Pre-Processing: Task Data

In [4]:
task_df = pd.read_csv(new_task_path, index_col='task_id')

tasks = task_df.index.tolist()
company_names = list(set(task_df['project_id']))
story_points = task_df['story_points'].to_dict()

task_df

,project_id,story_points,Mathematics.Linear Algebra,Mathematics.Differential Equations,Mathematics.Optimization Technique,Mathematics.Calculus,Mathematics.Combinatorics & Graph,Statistics & Probabilities.Statistics,Statistics & Probabilities.Probability & Sampling,Statistics & Probabilities.Bayessian Method,...,Data & Cloud Engineering.Data Security & Privacy,Data & Cloud Engineering.Distributed Systems,"Data & Cloud Engineering.Monitoring, Logging, Alerting",MLOPS.ML Lifecycle,MLOPS.ML Model Pipeline,MLOPS.Model Versioning,MLOPS.Automated Testing & Validation,MLOPS.Scalable Model Training,"MLOPS.Model Deployment, Monitoring, Logging",MLOPS.Ethical AI & Bias Mitigation
task_id,,,,,,,,,,,,,,,,,,,,,
T1,P5,8,5,1,4,4,4,4,3,5,...,4,1,4,2,3,5,4,5,5,5
T2,P1,1,5,5,5,1,5,3,1,2,...,3,4,3,1,1,1,4,1,3,3
T3,P5,2,5,4,5,1,5,4,5,2,...,2,4,3,5,4,2,3,4,2,5
T4,P1,3,3,1,2,1,5,4,3,1,...,4,1,1,4,1,4,1,3,2,1
T5,P2,5,1,4,5,5,2,3,5,5,...,3,5,1,3,1,5,5,5,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T296,P5,5,4,5,4,4,3,1,4,3,...,3,3,1,5,5,3,3,5,5,3
T297,P2,8,4,3,5,2,2,2,2,4,...,4,4,4,5,2,1,3,5,5,5
T298,P2,2,2,2,5,2,4,2,5,2,...,5,1,2,2,5,4,3,2,5,4


## 1.3. Group the task data by company/project

In [5]:
# convert to dictionary each company and its task
company_tasks = {}

for company in company_names:
  company_tasks[company] = task_df[task_df['project_id'] == company].index.tolist()

# sort the company tasks from C1 to C5
company_tasks = dict(sorted(company_tasks.items()))

company_tasks_df = pd.DataFrame.from_dict(company_tasks, orient='index')
company_tasks_df

,0,1,2,3,4,5,6,7,8,9,...,56,57,58,59,60,61,62,63,64,65
P1,T2,T4,T14,T21,T26,T29,T30,T36,T38,T40,...,None,None,None,None,None,None,None,None,None,None
P2,T5,T6,T9,T11,T19,T28,T32,T33,T44,T46,...,T260,T265,T274,T279,T283,T291,T293,T297,T298,T299
P3,T10,T23,T24,T25,T34,T48,T53,T58,T59,T68,...,T285,T286,T287,T289,T300,None,None,None,None,None
P4,T15,T20,T35,T37,T39,T43,T56,T57,T61,T70,...,None,None,None,None,None,None,None,None,None,None
P5,T1,T3,T7,T8,T12,T13,T16,T17,T18,T22,...,T241,T242,T243,T268,T276,T280,T282,T288,T296,None


## 1.4. Pre-Processing: Competency Assesment

first, create RCD-ACD Dataframe

### 1.4.1 RCD Dataframe

In [6]:
rcd_df = task_df.drop(columns=['project_id', 'story_points'])
rcd_df.head()

,Mathematics.Linear Algebra,Mathematics.Differential Equations,Mathematics.Optimization Technique,Mathematics.Calculus,Mathematics.Combinatorics & Graph,Statistics & Probabilities.Statistics,Statistics & Probabilities.Probability & Sampling,Statistics & Probabilities.Bayessian Method,Statistics & Probabilities.Hypothesis Testing,Statistics & Probabilities.A/B Testing,...,Data & Cloud Engineering.Data Security & Privacy,Data & Cloud Engineering.Distributed Systems,"Data & Cloud Engineering.Monitoring, Logging, Alerting",MLOPS.ML Lifecycle,MLOPS.ML Model Pipeline,MLOPS.Model Versioning,MLOPS.Automated Testing & Validation,MLOPS.Scalable Model Training,"MLOPS.Model Deployment, Monitoring, Logging",MLOPS.Ethical AI & Bias Mitigation
task_id,,,,,,,,,,,,,,,,,,,,,
T1,5,1,4,4,4,4,3,5,1,1,...,4,1,4,2,3,5,4,5,5,5
T2,5,5,5,1,5,3,1,2,2,4,...,3,4,3,1,1,1,4,1,3,3
T3,5,4,5,1,5,4,5,2,4,1,...,2,4,3,5,4,2,3,4,2,5
T4,3,1,2,1,5,4,3,1,1,2,...,4,1,1,4,1,4,1,3,2,1
T5,1,4,5,5,2,3,5,5,3,5,...,3,5,1,3,1,5,5,5,2,2


In [7]:
rcd_t1 = rcd_df.loc[['T1']]

### 1.4.2 ACD Dataframe

In [8]:
# create a copy of the original DataFrame
acd_df = employee_skills_df.copy()
acd_df

,Mathematics.Linear Algebra,Mathematics.Differential Equations,Mathematics.Optimization Technique,Mathematics.Calculus,Mathematics.Combinatorics & Graph,Statistics & Probabilities.Statistics,Statistics & Probabilities.Probability & Sampling,Statistics & Probabilities.Bayessian Method,Statistics & Probabilities.Hypothesis Testing,Statistics & Probabilities.A/B Testing,...,Data & Cloud Engineering.Data Security & Privacy,Data & Cloud Engineering.Distributed Systems,"Data & Cloud Engineering.Monitoring, Logging, Alerting",MLOPS.ML Lifecycle,MLOPS.ML Model Pipeline,MLOPS.Model Versioning,MLOPS.Automated Testing & Validation,MLOPS.Scalable Model Training,"MLOPS.Model Deployment, Monitoring, Logging",MLOPS.Ethical AI & Bias Mitigation
employee_id,,,,,,,,,,,,,,,,,,,,,
Talent 1,1,1,2,1,2,3,3,2,2,2,...,1,1,1,2,1,1,1,1,1,2
Talent 2,4,4,4,4,3,5,4,3,3,3,...,3,2,1,1,1,1,1,1,1,1
Talent 3,3,2,2,3,2,3,3,2,2,3,...,3,3,3,3,2,3,2,3,3,2
Talent 4,3,2,3,2,3,4,3,2,4,2,...,3,1,1,2,2,2,2,2,2,2
Talent 5,2,1,3,1,1,1,2,1,1,2,...,2,1,1,3,2,5,2,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Talent 107,1,4,4,3,4,3,4,4,4,4,...,5,5,5,3,5,3,4,3,4,2
Talent 108,3,3,3,3,4,2,2,2,2,3,...,4,3,4,2,3,3,2,2,3,2
Talent 109,5,5,4,5,3,4,4,3,3,3,...,5,5,5,5,5,5,5,5,5,5


### 1.4.3 Fit the Data

In [10]:
ca = CompetencyAssessment(rcd_t1, acd_df)
qs, info = ca.fit()

In [11]:
qs

{'Talent 1': {'T1': [0.0717948717948718,
   -2.0358974358974358,
   -0.03021696252465483,
   'Under-Qualified']},
 'Talent 2': {'T1': [0.2205128205128205,
   -1.4256410256410255,
   -0.018540433925049307,
   'Under-Qualified']},
 'Talent 3': {'T1': [0.2153846153846153,
   -1.3743589743589737,
   -0.01783037475345167,
   'Under-Qualified']},
 'Talent 4': {'T1': [0.2564102564102564,
   -1.3435897435897433,
   -0.016725838264299797,
   'Under-Qualified']},
 'Talent 5': {'T1': [0.15897435897435896,
   -2.169230769230769,
   -0.030927021696252464,
   'Under-Qualified']},
 'Talent 6': {'T1': [0.2717948717948718,
   -1.1282051282051277,
   -0.013175542406311632,
   'Under-Qualified']},
 'Talent 7': {'T1': [0.5999999999999999,
   -0.5589743589743588,
   0.0006311637080867862,
   'Qualified']},
 'Talent 8': {'T1': [1.143589743589743,
   -0.2512820512820512,
   0.013727810650887566,
   'Qualified']},
 'Talent 9': {'T1': [0.32307692307692315,
   -1.6153846153846145,
   -0.01988165680473371,
   'U

In [12]:
score = ca.rank_MSG(qs)
score

{'Talent 1': {'T1': -0.03021696252465483},
 'Talent 2': {'T1': -0.018540433925049307},
 'Talent 3': {'T1': -0.01783037475345167},
 'Talent 4': {'T1': -0.016725838264299797},
 'Talent 5': {'T1': -0.030927021696252464},
 'Talent 6': {'T1': -0.013175542406311632},
 'Talent 7': {'T1': 0.0006311637080867862},
 'Talent 8': {'T1': 0.013727810650887566},
 'Talent 9': {'T1': -0.01988165680473371},
 'Talent 10': {'T1': 0.011203155818540434},
 'Talent 11': {'T1': -0.00962524654832347},
 'Talent 12': {'T1': -0.014595660749506896},
 'Talent 13': {'T1': -0.01175542406311637},
 'Talent 14': {'T1': -0.012544378698224844},
 'Talent 15': {'T1': -0.021380670611439844},
 'Talent 16': {'T1': -0.00930966469428008},
 'Talent 17': {'T1': -0.03566074950690336},
 'Talent 18': {'T1': -0.02666666666666667},
 'Talent 19': {'T1': -0.00023668639053254183},
 'Talent 20': {'T1': -0.023037475345167653},
 'Talent 21': {'T1': 0.01088757396449704},
 'Talent 22': {'T1': -0.02011834319526627},
 'Talent 23': {'T1': -0.016568

In [13]:
info['weight']

{'Mathematics.Linear Algebra': task_id
 T1    0.025641
 dtype: float64,
 'Mathematics.Differential Equations': task_id
 T1    0.005128
 dtype: float64,
 'Mathematics.Optimization Technique': task_id
 T1    0.020513
 dtype: float64,
 'Mathematics.Calculus': task_id
 T1    0.020513
 dtype: float64,
 'Mathematics.Combinatorics & Graph': task_id
 T1    0.020513
 dtype: float64,
 'Statistics & Probabilities.Statistics': task_id
 T1    0.020513
 dtype: float64,
 'Statistics & Probabilities.Probability & Sampling': task_id
 T1    0.015385
 dtype: float64,
 'Statistics & Probabilities.Bayessian Method': task_id
 T1    0.025641
 dtype: float64,
 'Statistics & Probabilities.Hypothesis Testing': task_id
 T1    0.005128
 dtype: float64,
 'Statistics & Probabilities.A/B Testing': task_id
 T1    0.005128
 dtype: float64,
 'Statistics & Probabilities.CUPED': task_id
 T1    0.025641
 dtype: float64,
 'Data Structures & Algorithms.Complexity Analysis': task_id
 T1    0.010256
 dtype: float64,
 'Data St

# 2. Construct the Model

In [ ]:
model = cp_model.CpModel()

# 3. Build the Decision Variable

We have 3 sets
$$
sets=\begin{cases}I\:&,\:set\:of\:task\\ J&,\:set\:of\:employee\\ K&,\:set\:of\:project\end{cases}
$$

then we have parameters, scalars, and data structures. so let:
$$
i=\:task \:i \\
j=employee \:j\\
k=projects \:k\\
s_i=story\:points\:of\:task \:i \\
e_{ij}=similarity\:skills\:of\:employee\:j\:for\:task\:i \\
$$



Decision Variable

$$
x_{ijk}=Binary\:variable\:indicating\:whether\:employee\:j\:is\:assigned\:to\:task\:k\:for\:day\:i
$$
$$
y_{jk}=Binary\:variable\:indicating\:whether\:employee\:j\:is\:assigned\:to\:any\:task\:from\:company\:k
$$

In [ ]:
max_employee_workload = 20

In [ ]:
# Create decision variables for x and y
x = {}
for k, task in company_tasks.items():
    for i in task:
        for j in employees:
            x[(i, j, k)] = model.NewIntVar(0, 1, f'x_{i}_{j}_{k}')

# decision variable y represent cardinility of each employee and company
y = {}
for j in employees:
    for k in company_tasks.keys():
        y[(j, k)] = model.NewIntVar(0, 1, f'y_{j}_{k}')

# decision variables max_workload
max_workload = model.NewIntVar(0, max_employee_workload, 'max_workload')

In [ ]:
print(x)
print(y)

# 4. Subject to the Constraint

## Constraint 1: Each task is assigned to one employee
$$
\sum _{j\in J}\:x_{ijk}\:=\:1 \quad \forall i \in k, j \in J, k \in K
$$

In [ ]:
# constraint 1: each task assigned to one talent
for k, task in company_tasks.items():
    for i in task:
        model.Add(sum(x[(i, j, k)] for j in employees) == 1)

## Constraint 2: Each employee works for one company at a time
$$
Pre-Constraint\:2:\sum _{i\in I_k}x_{ijk}>0\:\rightarrow \:y_{jk}=1\:\forall j\in J,\:\forall k\in K\:
$$

In [ ]:
# pre-processing constraint 2
for j in employees:
    for k, task in company_tasks.items():
        # Create a temporary list to hold the sum of x[i][j][k] for all i
        temp_sum = []
        for i in task:
            temp_sum.append(x[(i, j, k)])
        # Add a constraint to the model: y[j][k] is 1 if the sum of x[i][j][k] for all i is > 0, and 0 otherwise
        model.Add(sum(temp_sum) > 0).OnlyEnforceIf(y[(j, k)])
        model.Add(sum(temp_sum) <= 0).OnlyEnforceIf(y[(j, k)].Not())

$$
Constraint\:2:\sum _{k\in K}y_{jk}\le 1\:\forall j\in J,k\in K\:
$$

In [ ]:
# create constraint 2: each employee can only work on one task
for j in employees:
    # The sum of y[j][k] for all companies (k) should be <= 1
    model.Add(sum(y[(j, k)] for k in company_tasks.keys()) <= 1)

## Constraint 3: Employee workload doesn't exceed the capacity
$$
Constraint\:3:\:\sum _{j\in J}s_i\cdot x_{ijk}\le 20\:\forall i\in k,j\in J, k\in K\:
$$

In [ ]:
# constraint 3: employee workload doesn't exceed the capacity
for j in employees:
  model.Add(sum(story_points[i] * x[i, j, k] for k, tasks in company_tasks.items() for i in tasks) <= max_employee_workload)

## Constraint 4: Maximum workload is greater than or equal to the workload of each employee
$$
Constraint\:4:\:maxworkload\ge \sum _{i,\:k}s_i\cdot x_{i,j,k},\:\forall j\in J\:\:
$$

In [ ]:
# constraint 4: max_workload is greater than or equal to the workload of each employee
for j in employees:
    model.Add(max_workload >= sum(story_points[i] * x[i, j, k] for k, tasks in company_tasks.items() for i in tasks))

# 5. Single Objective 1: Minimize the Idle Employee

## 5.1. Set the Objective Model

## Single Objective Minimize Idle Employee
$$
min.\:I_j=\sum _{j\in \:J}\:\left(1\:-\:\sum _{k\in \:K}\:y_{jk}\right)\quad \: \forall j\in J, k\in K\quad \tag{1}
$$

In [ ]:
# objective 1
I = []

for j in employees:
  obj1 = 1 - sum(y[j, k] for k in company_tasks.keys())
  I.append(obj1)

I_total_idle_employee = sum(I)

# single objective 1
model.Minimize(I_total_idle_employee)

## 5.2. Solve the Model

In [ ]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [ ]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
  print('Solution Found!')
  print(f'Objective Value: {solver.ObjectiveValue()}\n')

  print(f'Total Idle Employees: {solver.Value(I_total_idle_employee)}')

  x_hat = {}

  for j in employees:
    task = []
    sim = []
    sp = 0
    wasted_sp = 0
    comp = []

    for k, tasks in company_tasks.items():
      for i in tasks:
        if solver.Value(x[i, j, k]) == 1:
          print(f'Task {i} assigned to Employee {j}')
          print(f'Company\t\t\t: {k}')
          print(f'Story Points\t\t: {story_points[i]}')
          print(f"Similarity score\t: {cosine_similarity_score[i, j]:.10f}\n")

          task.append(i)
          sim.append(cosine_similarity_score[i, j])
          comp.append(k)
          sp += story_points[i]

    if sp > 0:
      wasted_sp = max_employee_workload - sp
      x_hat[j] = comp, task, sp, wasted_sp, sim

else:
  print('No Solution Found!')
  x_hat = {}

## 5.3. Print the Result

In [ ]:
# Convert dictionary to DataFrame
result1 = pd.DataFrame([(key, value[0], value[1], value[2], value[3], value[4]) for key, value in x_hat.items()],
                      columns=['employee', 'company', 'assigned_task', 'sum_sp', 'wasted_sp', 'similarity_score'])

# Set 'company' as index
result1.set_index('employee', inplace=True)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

result1

In [ ]:
total_employee = len(employees)
total_sp = sum(story_points.values())
total_active_employee = len(set(employee for employee in x_hat.keys()))
total_active_sp = sum(value[2] for value in x_hat.values())
total_idle_employee = total_employee - total_active_employee
total_wasted_sp = total_sp - total_active_sp

print(f'Total Employee\t\t\t: {total_employee}')
print(f'Total Active Employee\t\t: {total_active_employee}\t{(total_active_employee/total_employee)*100:.2f}%')
print(f'Total Idle Employee\t\t: {total_idle_employee}\t{(total_idle_employee/total_employee)*100:.2f}%\n')
print(f'Total Story Points\t\t: {total_sp}')
print(f'Total Active Story Points\t: {total_active_sp}\t{(total_active_sp/total_sp)*100:.2f}%')
print(f'Total Wasted Story Points\t: {total_wasted_sp}\t{(total_wasted_sp/total_sp)*100:.2f}%\n')

In [ ]:
# make boxplot for objective 1 variable from the similarity score
similarity_score1 = result1['similarity_score'].explode().reset_index(drop=True)
similarity_score1.plot(kind='box')
plt.title('Similarity Score Boxplot')
plt.show()

# 6. Single Objective 2: Maximize the Similarity Error

## 6.1. Set the Objective Model

## Single Objective Maximize Similarity Score
$$
max.\:E_{ij}=\sum_{i\in I}\:\sum _{j\in J}\:e_{ij}\cdot x_{ijk}\quad \:\:\forall \:i\in k, \:j \in J, \:k \in K\quad\tag{2}
$$

In [ ]:
# objective 2
# E_total_similarity_score = 0

for k, tasks in company_tasks.items():
  E_total_similarity_score = sum(cosine_similarity_score[i, j] * x[i, j, k] for i in tasks for j in employees)


model.Maximize(E_total_similarity_score)


# E_total_similarity_score = []

# for k, tasks in company_tasks.items():
#   for j in employees:
#     obj2 = sum(cosine_similarity_score[i, j] * x[i, j, k] for i in tasks for j in employees)
#     E_total_similarity_score.append(obj2)

# model_2.Maximize(E_total_similarity_score)

## 6.2. Solve the Model

In [ ]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [ ]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
  print('Solution Found!')
  print(f'Objective Value: {solver.ObjectiveValue()}\n')

  print(f'Total Similarity Score: {solver.Value(E_total_similarity_score)}\n')

  x_hat = {}

  for j in employees:
    task = []
    sim = []
    sp = 0
    wasted_sp = 0
    comp = []

    for k, tasks in company_tasks.items():
      for i in tasks:
        if solver.Value(x[i, j, k]) == 1:
          print(f'Task {i} assigned to Employee {j}')
          print(f'Company\t\t\t: {k}')
          print(f'Story Points\t\t: {story_points[i]}')
          print(f"Similarity score\t: {cosine_similarity_score[i, j]:.10f}\n")

          task.append(i)
          sim.append(cosine_similarity_score[i, j])
          comp.append(k)
          sp += story_points[i]

    if sp > 0:
      wasted_sp = max_employee_workload - sp
      x_hat[j] = comp, task, sp, wasted_sp, sim

else:
  print('No Solution Found!')
  x_hat = {}

## 6.3. Print the Result

In [ ]:
# Convert dictionary to DataFrame
result2 = pd.DataFrame([(key, value[0], value[1], value[2], value[3], value[4]) for key, value in x_hat.items()],
                      columns=['employee', 'company', 'assigned_task', 'sum_sp', 'wasted_sp', 'similarity_score'])

# Set 'company' as index
result2.set_index('employee', inplace=True)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

result2

In [ ]:
total_employee = len(employees)
total_sp = sum(story_points.values())
total_active_employee = len(set(employee for employee in x_hat.keys()))
total_active_sp = sum(value[2] for value in x_hat.values())
total_idle_employee = total_employee - total_active_employee
total_wasted_sp = total_sp - total_active_sp

print(f'Total Employee\t\t\t: {total_employee}')
print(f'Total Active Employee\t\t: {total_active_employee}\t{(total_active_employee/total_employee)*100:.2f}%')
print(f'Total Idle Employee\t\t: {total_idle_employee}\t{(total_idle_employee/total_employee)*100:.2f}%\n')
print(f'Total Story Points\t\t: {total_sp}')
print(f'Total Active Story Points\t: {total_active_sp}\t{(total_active_sp/total_sp)*100:.2f}%')
print(f'Total Wasted Story Points\t: {total_wasted_sp}\t{(total_wasted_sp/total_sp)*100:.2f}%\n')

In [ ]:
# make boxplot for objective 2 variable from the similarity score
similarity_score2 = result2['similarity_score'].explode().reset_index(drop=True)
similarity_score2.plot(kind='box')
plt.title('Similarity Score Boxplot')
plt.show()

# 7. Single Objective 3: Balancing Workload

## 7.1 Set The Objective Model

## Single Objective Balancing Workload Employee

In [ ]:
model.Minimize(max_workload)

## 7.2 Solve the Model

In [ ]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

## 7.3 Print the Result

In [ ]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
  print('Solution Found!')
  print(f'Objective Value: {solver.ObjectiveValue()}\n')

  print(f'Total Similarity Score: {solver.Value(E_total_similarity_score)}\n')

  x_hat = {}

  for j in employees:
    task = []
    sim = []
    sp = 0
    wasted_sp = 0
    comp = []

    for k, tasks in company_tasks.items():
      for i in tasks:
        if solver.Value(x[i, j, k]) == 1:
          print(f'Task {i} assigned to Employee {j}')
          print(f'Company\t\t\t: {k}')
          print(f'Story Points\t\t: {story_points[i]}')
          print(f"Similarity score\t: {cosine_similarity_score[i, j]:.10f}\n")

          task.append(i)
          sim.append(cosine_similarity_score[i, j])
          comp.append(k)
          sp += story_points[i]

    if sp > 0:
      wasted_sp = max_employee_workload - sp
      x_hat[j] = comp, task, sp, wasted_sp, sim

else:
  print('No Solution Found!')
  x_hat = {}

In [ ]:
# Convert dictionary to DataFrame
result3 = pd.DataFrame([(key, value[0], value[1], value[2], value[3], value[4]) for key, value in x_hat.items()],
                      columns=['employee', 'company', 'assigned_task', 'sum_sp', 'wasted_sp', 'similarity_score'])

# Set 'company' as index
result3.set_index('employee', inplace=True)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

result3

In [ ]:
total_employee = len(employees)
total_sp = sum(story_points.values())
total_active_employee = len(set(employee for employee in x_hat.keys()))
total_active_sp = sum(value[2] for value in x_hat.values())
total_idle_employee = total_employee - total_active_employee
total_wasted_sp = total_sp - total_active_sp

print(f'Total Employee\t\t\t: {total_employee}')
print(f'Total Active Employee\t\t: {total_active_employee}\t{(total_active_employee/total_employee)*100:.2f}%')
print(f'Total Idle Employee\t\t: {total_idle_employee}\t{(total_idle_employee/total_employee)*100:.2f}%\n')
print(f'Total Story Points\t\t: {total_sp}')
print(f'Total Active Story Points\t: {total_active_sp}\t{(total_active_sp/total_sp)*100:.2f}%')
print(f'Total Wasted Story Points\t: {total_wasted_sp}\t{(total_wasted_sp/total_sp)*100:.2f}%\n')

In [ ]:
# make boxplot for objective 3 variable from the similarity score
similarity_score3 = result3['similarity_score'].explode().reset_index(drop=True)
similarity_score3.plot(kind='box')
plt.title('Similarity Score Boxplot')
plt.show()

# Boxplot Visualization 3 Single Objective

In [ ]:
# merge all boxplot in one graph
plt.figure(figsize=(10, 5))
plt.boxplot([similarity_score1, similarity_score2, similarity_score3], labels=['Objective 1', 'Objective 2', 'Objective 3'])
plt.title('Similarity Score Boxplot')
plt.show()

# 8. Multi-Objective Approach

## 8.1. Set The Objective Model

## Multi Objective Approach

$$
min.\:W=\alpha \cdot min.\:I_j+\beta \cdot max.\:E_{ij} \quad \tag{3}
$$

In [ ]:
alpha = 0.1
beta = 0.1
theta = 0.8

In [ ]:
# objective 1
# I = []

# for j in employees:
#   obj1 = 1 - sum(y[j, k] for k in company_tasks.keys())
#   I.append(obj1)

# I_total_idle_employee = sum(I)

# # objective 2
# for k, tasks in company_tasks.items():
#   E_total_similarity_score = sum(cosine_similarity_score[i, j] * x[i, j, k] for i in tasks for j in employees)

model.Minimize((alpha * I_total_idle_employee) + (beta * E_total_similarity_score) + (theta * max_workload))

## 8.2. Solve the Model

In [ ]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [ ]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
  print('Solution Found!')
  print(f'Objective Value: {solver.ObjectiveValue()}\n')

  print(f'Total Idle Employees: {solver.Value(I_total_idle_employee)}')
  print(f'Total Similarity Score: {solver.Value(E_total_similarity_score)}\n')

  x_hat = {}

  for j in employees:
    task = []
    sim = []
    sp = 0
    wasted_sp = 0
    comp = []

    for k, tasks in company_tasks.items():
      for i in tasks:
        if solver.Value(x[i, j, k]) == 1:
          print(f'Task {i} assigned to Employee {j}')
          print(f'Company\t\t\t: {k}')
          print(f'Story Points\t\t: {story_points[i]}')
          print(f"Similarity score\t: {cosine_similarity_score[i, j]:.10f}\n")

          task.append(i)
          sim.append(cosine_similarity_score[i, j])
          comp.append(k)
          sp += story_points[i]

    if sp > 0:
      wasted_sp = max_employee_workload - sp
      x_hat[j] = comp, task, sp, wasted_sp, sim

else:
  print('No Solution Found!')
  x_hat = {}

## 8.3. Print the Result

In [ ]:
# Convert dictionary to DataFrame
result4 = pd.DataFrame([(key, value[0], value[1], value[2], value[3], value[4]) for key, value in x_hat.items()],
                      columns=['employee', 'company', 'assigned_task', 'sum_sp', 'wasted_sp', 'similarity_score'])

# Set 'company' as index
result4.set_index('employee', inplace=True)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

result4

In [ ]:
total_employee = len(employees)
total_sp = sum(story_points.values())
total_active_employee = len(set(employee for employee in x_hat.keys()))
total_active_sp = sum(value[2] for value in x_hat.values())
total_idle_employee = total_employee - total_active_employee
total_wasted_sp = total_sp - total_active_sp

print(f'Total Employee\t\t\t: {total_employee}')
print(f'Total Active Employee\t\t: {total_active_employee}\t{(total_active_employee/total_employee)*100:.2f}%')
print(f'Total Idle Employee\t\t: {total_idle_employee}\t{(total_idle_employee/total_employee)*100:.2f}%\n')
print(f'Total Story Points\t\t: {total_sp}')
print(f'Total Active Story Points\t: {total_active_sp}\t{(total_active_sp/total_sp)*100:.2f}%')
print(f'Total Wasted Story Points\t: {total_wasted_sp}\t{(total_wasted_sp/total_sp)*100:.2f}%\n')

In [ ]:
# Data for employees
employee_labels = ['Total Employees', 'Total Active Employees', 'Total Idle Employees']
employee_values = [total_employee, total_active_employee, total_idle_employee]

# Data for story points
sp_labels = ['Total Story Points', 'Total Active Story Points', 'Total Wasted Story Points']
sp_values = [total_sp, total_active_sp, total_wasted_sp]

# Create subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot for employees
ax1.barh(employee_labels, employee_values, color='skyblue')
ax1.set_xlabel('Count')
ax1.set_title('Summary of Employees')
ax1.grid(axis='x', linestyle='--', alpha=0.7)

# Plot for story points
ax2.barh(sp_labels, sp_values, color='lightgreen')
ax2.set_xlabel('Count')
ax2.set_title('Summary of Story Points')
ax2.grid(axis='x', linestyle='--', alpha=0.7)

# Show plot
plt.tight_layout()
plt.show()

In [ ]:
# make boxplot for multi objective variable from the similarity score
similarity_score_multi = result4['similarity_score'].explode().reset_index(drop=True)
similarity_score_multi.plot(kind='box')
plt.title('Similarity Score Boxplot')
plt.show()

# Boxplot Visualization

In [ ]:
# merge all boxplot in one graph
plt.figure(figsize=(10, 5))
plt.boxplot([similarity_score1, similarity_score2, similarity_score3, similarity_score_multi],
            labels=['Obj 1: Min Idle Emp', 'Obj 2: Max Similarity Score', 'Obj 3: Balancing the Workload', 'Multi Objective'])
plt.title('Similarity Score Boxplot')
plt.xticks(rotation=15)
plt.show()

# 9. Strategic Analysis

## Single Objective -

In [ ]:
## Single Objective -